In [1]:
!pip install pdf2image

In [2]:
######################################### IMPORTING PACAKGES #############################
# Basic ML Packages
from scipy import spatial
import pandas as pd
import math
import os
import json
import numpy as np
import string

import warnings
warnings.filterwarnings("ignore")

# PDF text extraction
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter

# Others
import string
import re
from pprint import pprint
from tqdm.notebook import tqdm
import io

# Text pre-processing (Tokenization, Stemming, Lemmatization)
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

# Pdf Extraction Model
import spacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=['ner'])

#Gensim stopwords
import gensim
from gensim.parsing.preprocessing import remove_stopwords
stopwords = gensim.parsing.preprocessing.STOPWORDS

# Train Test Split
from sklearn.model_selection import train_test_split

# Tf-Idf Vectorization
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import numpy as np
import pandas as pd
import PyPDF2
import tabula
from tabula import read_pdf
from tabulate import tabulate
import io
import camelot
from functools import reduce
from pdfminer.high_level import extract_text

DATA_FOLDER = "datasets/real_estate"

[nltk_data] Downloading package punkt to C:\Users\Chen
[nltk_data]     Wei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Chen
[nltk_data]     Wei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Chen
[nltk_data]     Wei\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import fitz

In [26]:
keywords = ['scope 1', 'scope 2', 'scope 3', 'location-based', 'market-based']
keywords = [x.lower() for x in keywords]
keywords

['scope 1', 'scope 2', 'scope 3', 'location-based', 'market-based']

In [27]:
DATA_FOLDER = 'datasets/real_estate'
list_dataset = os.listdir(DATA_FOLDER)
pdfFile = open(DATA_FOLDER + '/British Land 2022 Sustainability Report.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFile)
totpages = pdfReader.numPages


page_with_keywords = []
for p in range(pdfReader.numPages):
    text = pdfReader.pages[p].extract_text().lower()
    if any(x in text for x in keywords):
        page_with_keywords.append(p + 1)

Xref table not zero-indexed. ID numbers for objects will be corrected.


In [28]:
page_with_keywords

[4, 7, 30, 32, 33, 34, 35, 36, 37, 38, 44, 63, 71, 72, 74, 75, 76, 88]

In [29]:
## Table check
table_pages = []
for i in page_with_keywords:
    pdf = read_pdf(DATA_FOLDER + '/British Land 2022 Sustainability Report.pdf', pages=i, stream=True, pandas_options={'header':'None'}, multiple_tables=True)
    if len(pdf) > 0:
        table_pages.append(i)

In [30]:
table_pages

[7, 30, 32, 33, 34, 35, 36, 37, 38, 44, 63, 72, 75, 76]

In [68]:
def get_images_keyword(file_path, keywords, output_path, table_only=False):
    dataset_list = os.listdir(file_path)
    for file in dataset_list:
        #reading pdf file to filter keywords
        pdfFile = open(file_path + '/' + file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFile)
        totpages = pdfReader.numPages
        
        print("Starting with file: " + file)
        page_with_keywords = []
        for p in range(pdfReader.numPages):
            text = pdfReader.pages[p].extract_text().lower()
            if any(x in text for x in keywords):
                if (p+1) not in page_with_keywords:
                    page_with_keywords.append(p + 1)
        
        ## Filter for only tables.
        if table_only:
            table_pages = []
            for i in page_with_keywords:
                pdf = read_pdf(file_path + '/' + file, pages=i, stream=True, pandas_options={'header':'None'}, multiple_tables=True)
                if len(pdf) > 0:
                    table_pages.append(i)
            page_with_keywords = table_pages
        
        ##Extract images
        for i in page_with_keywords:
            pdf2image.convert_from_path(file_path + '/' + file, output_folder = output_path, fmt='png', 
                                       first_page = i, last_page = i, output_file = str(file) + str(i))
        
        print('Finished with file: ' + file)
    return ""
    

In [69]:
get_images_keyword(DATA_FOLDER, keywords, 'real_estate_images', table_only = True)

Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf
Befimmo 2021 Sustainability Report.pdf


Xref table not zero-indexed. ID numbers for objects will be corrected.


Finished with file: Befimmo 2021 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
British Land 2022 Sustainability Report.pdf
Finished with file: British Land 2022 Sustainability Report.pdf
CDL 2022 Sustainability Report.pdf
CDL 20

Xref table not zero-indexed. ID numbers for objects will be corrected.


Finished with file: CICT 2021 Annual Report.pdf
First Capital REIT ESG Roadmap.pdf
First Capital REIT ESG Roadmap.pdf
Finished with file: First Capital REIT ESG Roadmap.pdf
GPT 2021 Sustainability Report.pdf
GPT 2021 Sustainability Report.pdf
GPT 2021 Sustainability Report.pdf
GPT 2021 Sustainability Report.pdf
GPT 2021 Sustainability Report.pdf
GPT 2021 Sustainability Report.pdf
GPT 2021 Sustainability Report.pdf


...

Xref table not zero-indexed. ID numbers for objects will be corrected.


Finished with file: Klepierre 2021 URD.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf
Prologis 2021-22 ESG Report.pdf


Xref table not zero-indexed. ID numbers for objects will be corrected.


Finished with file: Prologis 2021-22 ESG Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Swire Pacific 2021 Sustainability Report.pdf
Finished with file: Swire Pacific 2021 Sustainability Report.pdf


''